In [1]:
import pandas as pd
import random
import numpy as np
from scipy.spatial import distance_matrix
import itertools
import copy
import time

In [2]:
def generate_dm(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)

    temp = df[2].to_numpy() // 2
    temp = temp * np.ones((200, 200))
    dm = dm + temp + temp.T
    dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf

    if show:
        df_dist = pd.DataFrame(dm)
        # display(df_dist)
    return dm


def get_random_solution():
    return random.sample([i for i in range(0, 200)], 100)


def calulate_total_cost(path, dm):
    total = 0
    nr = len(path)
    for idx, node in enumerate(path):
        total += dm[node][path[(idx + 1) % nr]]
    return total

In [3]:
tsp = pd.read_csv('TSPC.csv', sep=';', header=None)

In [149]:
def calc_sub_path_cost(sub_path, dm):
    total = 0
    for idx, node in enumerate(sub_path[:-1]):
        total += dm[node][sub_path[idx+1]]
    return total

def destroy(path, total_cost, m=5, to_remove=20):

    sub_paths = []
    total = 0 
    factor = m-1
    for i in range(len(path)):
        if i+m > len(path):
            sub_path = path[i:] + path[0:(i+m)%100]
            cost = calc_sub_path_cost(sub_path, dm)
            total += cost/(factor*total_cost)
            sub_paths.append((cost/(factor*total_cost), path[i]))
        else:
            sub_path = path[i:i+m]
            cost = calc_sub_path_cost(sub_path, dm)
            total += cost/(factor*total_cost)
            sub_paths.append((cost/(factor*total_cost), path[i]))
            
    total_removed = 0
    while total_removed < to_remove:
        d = random.random()
        iterator = 0
        for cost, node in sub_paths:
            iterator += cost
            if iterator > d:
                if node not in path:
                    continue
                idx = path.index(node)
                for i in range(1, factor):
                    if path[(idx+i)%100] != '_':
                        path[(idx+i)%100] = '_' 
                        total_removed += 1
                sub_paths.remove((cost, node))
                break

    return path

In [150]:
def greedy(node1, node2, lenght, path):
    part = [node1, node2]
    not_in_path = list(set(range(200)) - set(path))
    
    for _ in range(lenght):
        moves = []
        for idx, node1 in enumerate(part):
            node2 = part[(idx+1)%len(part)]
            for new_node in not_in_path:
                new_dist = dm[node1][new_node] + dm[node2][new_node] - dm[node1][node2]
                moves.append((new_dist, new_node, idx+1))
        moves.sort(key = lambda x: x[0])
        part.insert(moves[0][2], moves[0][1])
        not_in_path.remove(moves[0][1])

    return part
        

def repair(path):
    if path[0] == '_':
        while path[0] == '_':
            path.append('_')
            path.pop(0)
#         print(path)
    if path[-1] == '_':
        path.append(path[0])
        mem = path[0]

    while '_' in path:
        idx1 = path.index('_')
        idx2 = copy.deepcopy(idx1)
        try:
            while path[idx2] == '_':
                idx2+=1
            new_part = greedy(path[idx1-1], path[idx2], idx2-idx1, path)

            path[idx1-1:idx2+1] = new_part
        except Exception as e:
            print(e, idx2, path)
            break

    if len(path) > 100:
        if path[0] == mem:
            path.pop(0)
#         print(path)
    return path


In [151]:
def gen_node_exchange_moves(path):
    not_selected = list(set([i for i in range(200)]) - set(path))
    node_moves = []
    for idx, node in enumerate(path):
        for new_node in not_selected:
            new_cost = dm[path[idx-1]][new_node] + dm[path[(idx+1)%100]][new_node] # two new edges
            new_cost -= (dm[path[idx-1]][node] + dm[path[(idx+1)%100]][node]) #
            node_moves.append((new_cost, 'n', idx, new_node)) #cost, type, idx of old move, new_node
    return node_moves


def gen_edge_exchange_moves(path):
    edge_moves = []
    for idx1, idx2 in all_posibble_comb:
        if (idx2+1)%100 != idx1:
            new_cost = dm[path[idx1]][path[(idx2+1)%100]] + dm[path[idx1-1]][path[idx2]] #new edges
            new_cost -= (dm[path[idx1-1]][path[idx1]] + dm[path[idx2]][path[(idx2+1)%100]]) #old edges
            edge_moves.append((new_cost, 'e', idx1, idx2))
    return edge_moves


def local_search(dm,all_combinations,path):
    run = True
    while run:
        run = False
        moves = gen_edge_exchange_moves(path) + gen_node_exchange_moves(path)
        moves.sort(key=lambda x:x[0])
        best_move = moves[0]
        if best_move[0] < 0:
            if best_move[1] == 'n':
                idx = best_move[2]
                new_node = best_move[3]
                path[idx] = new_node
                run = True
            else:
                idx1 = best_move[2]
                idx2 = best_move[3]
                path = path[:idx1] + path[idx1:idx2+1][::-1] + path[idx2+1:]
                run = True
    return path

In [152]:
dm = generate_dm(tsp)
all_posibble_comb = list(itertools.combinations([i for i in range(100)], 2))
# flag = False

In [165]:
def run(flag, m_pool=[5], to_remove_pool=[20], total_time=300):
    start = time.time()
    path = get_random_solution()
    path = local_search(dm, all_posibble_comb, path)
    i = 0
    while time.time() - start < total_time:
        if len(set(path)) != 100:
            print('error')
            break
        total_cost = calulate_total_cost(path, dm)
        
        m = random.choice(m_pool)
        to_remove = random.choice(to_remove_pool)

        new_path = destroy(copy.deepcopy(path), total_cost, m, to_remove)

        new_path = repair(new_path)
        if flag:
            new_path = local_search(dm, all_posibble_comb, new_path)

        if calulate_total_cost(new_path, dm) < total_cost:
            path = new_path
        i += 1
    return i, path
run(True)

(1050, 47469.0)

In [164]:
run(True, [3, 5, 7, 9, 11], [10, 20, 30, 40, 50], 300)

(695, 46850.0)

### Destroy
1. Generate all (in tis problem 100) sub_path's of the length m (for example [21, 88, 12, 44, 37]) and calculate `k`=cost(sub_path)/(`m`-1)*cost(path)
2. Until we remove at least `Z` nodes from the path:

    2.1 Chose at random subpath, end replace all nodes that are in this subpath from path with `_`; probabality of choosing=`k`.
    
### Repair
1. Until there are some `_` in path:

    1.1 Find old_sub_path which contains string of `_` (for example [21, `_`, `_`, `_`, 37])
    
    1.2 Create new_sub_path using greedy_cycle method with both side nodes as staring path, of the same length as old_sub_path.(for example [21, 55, 33, 37, 122])
    
    1.3 Replace old_sub_path with new_sub_path
    
### pseudo
1. path = Generate random solution 
2. path = local_search(path)
3. until time constrain

    3.1 get random m and Z
    
    3.2 new_path = destroy(path, m, Z)
    
    3.3 new_path = repair(path)
    
    3.4 if flag=True, run new_path = local_search(path)
    
    3.5 if cost(new_path)<cost(path): path=new_path